In [1]:
from libs.chatglm3 import ChatGLM3

In [2]:
MODEL_PATH = 'D:\Github\Repositories\models\huggingface\chatglm3-6b'
llm = ChatGLM3()
llm.load_model(MODEL_PATH)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
from langchain_community.document_loaders import DirectoryLoader

In [4]:
loader = DirectoryLoader('./static/books')
documents = loader.load()   

In [5]:
from langchain.text_splitter import CharacterTextSplitter

In [6]:
text_splitter = CharacterTextSplitter(chunk_size=2048, chunk_overlap=8)

In [7]:
splitted_documents = text_splitter.split_documents(documents)

In [8]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [9]:
encode_kwargs = {'normalize_embeddings': False}
model_kwargs = {'device': 'cuda:0'}
embeddings = HuggingFaceEmbeddings(model_name='shibing624/text2vec-base-chinese',model_kwargs=model_kwargs,encode_kwargs=encode_kwargs)

In [10]:
from langchain.vectorstores import Chroma

In [11]:
db = Chroma.from_documents(splitted_documents, persist_directory="VectorStore", embedding=embeddings)
db.persist()

In [12]:
from langchain.chains import RetrievalQA

In [13]:
retriever = db.as_retriever()

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""

In [15]:
custom_rag_prompt = PromptTemplate.from_template(template)


In [16]:
def format_docs(docs):
    return "\n\n".join(document.page_content for document in documents)

In [17]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [18]:
# qa = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type='stuff',
#     retriever=retriever
# )

In [19]:
rag_chain.invoke(' 医学难以解释的症状是什么？')

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
